In [1]:
import os, sys, time
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import joblib
#import pickle
import pandas
import torch
sys.path.append("/Users/tizianocausin/Desktop/backUp20240609/summer2025/ponce_lab/exp_set/python_scripts/svcca")
import cca_core
from sklearn.cross_decomposition import CCA
from torchvision.models.feature_extraction import (
    create_feature_extractor,
    get_graph_node_names,
)
from datetime import datetime
sys.path.append("/Users/tizianocausin/Desktop/backUp20240609/summer2025/ponce_lab/exp_set/python_scripts/src")
from dim_redu_anns.utils import get_relevant_output_layers
from alignment.CCA import CCA_loop_within_mod

In [ ]:
res_path = "/Users/tizianocausin/OneDrive - SISSA/data_repo/exp_set_res/silico"
model_name = "alexnet"
pooling = "maxpool"
num_components = 50
CCA_loop_within_mod(model_name, pooling, num_components, res_path)

17:56:02 stating layers {target_layer1} vs {target_layer2}
17:56:02 CCA already exists for features.4 vs features.0  at /Users/tizianocausin/OneDrive - SISSA/data_repo/exp_set_res/silico/cca_alexnet_maxpool/cca_alexnet_50_components_features.4_vs_features.0.pkl
17:56:02 stating layers {target_layer1} vs {target_layer2}
17:56:02 CCA already exists for features.7 vs features.0  at /Users/tizianocausin/OneDrive - SISSA/data_repo/exp_set_res/silico/cca_alexnet_maxpool/cca_alexnet_50_components_features.7_vs_features.0.pkl
17:56:02 stating layers {target_layer1} vs {target_layer2}
17:56:02 CCA already exists for features.7 vs features.4  at /Users/tizianocausin/OneDrive - SISSA/data_repo/exp_set_res/silico/cca_alexnet_maxpool/cca_alexnet_50_components_features.7_vs_features.4.pkl
17:56:02 stating layers {target_layer1} vs {target_layer2}
17:56:02 CCA already exists for features.9 vs features.0  at /Users/tizianocausin/OneDrive - SISSA/data_repo/exp_set_res/silico/cca_alexnet_maxpool/cca_ale